In [14]:
from boilerplate import setup_nb
%load_ext autoreload
%autoreload 2
device = setup_nb()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CUDA is available.
Changed working dir to /home/spandan/Projects/hubbard-transformer


In [15]:
from embedding import HubbardEmbedding
import torch
import einops as ein

In [16]:
N_PARAMS = 5
TOKEN_DIMS = 32
PARAM_DIMS = 16
MAX_LEN = 100
MIN_OCC = 0
MAX_OCC = 10
BATCH_SIZE = 32
model_dims = PARAM_DIMS + TOKEN_DIMS

In [17]:
embedding = HubbardEmbedding(TOKEN_DIMS, PARAM_DIMS, N_PARAMS, MAX_LEN).to(device)

In [18]:
(test_params := torch.randn(N_PARAMS, BATCH_SIZE).to(device))

tensor([[-4.7716e-01,  6.5343e-01,  1.9840e+00,  1.7509e-01,  5.3110e-01,
         -1.8015e+00,  1.4112e+00, -1.2091e-01,  1.0628e-01,  6.9504e-01,
         -1.3951e-01, -1.3992e+00,  1.9987e+00, -3.8545e-01,  1.7002e+00,
          4.3447e-02, -7.7260e-01, -1.9078e+00,  1.7566e+00,  1.1028e+00,
          2.6730e-01,  9.8881e-02,  5.1943e-01,  6.0680e-01, -8.8132e-01,
          6.2285e-01,  8.2657e-01, -1.2381e+00, -9.1094e-01, -5.4803e-01,
         -7.9872e-01, -4.2409e-01],
        [ 1.3855e-01,  1.2544e+00, -1.8954e+00, -1.8306e-01, -2.5580e-01,
          5.9590e-01, -1.4492e+00, -2.0904e+00, -3.0844e-01,  9.9123e-01,
         -9.8212e-01, -8.7414e-01, -1.1447e+00,  7.3516e-01, -1.5166e-03,
          1.8346e+00,  3.1560e-01, -1.0362e+00, -3.2777e-01, -1.6440e-01,
          4.5278e-01,  3.9598e-01,  7.7467e-01, -3.8708e-01, -3.7427e-01,
          1.1377e+00,  7.9152e-01,  5.6179e-01, -1.7029e+00, -7.1699e-01,
         -9.3843e-01,  2.0246e-01],
        [-1.5501e+00,  9.1469e-01,  5.78

In [19]:
(test_occupations := torch.randint(MIN_OCC, MAX_OCC, (MAX_LEN, BATCH_SIZE)).to(device))

tensor([[9, 7, 2,  ..., 2, 6, 2],
        [3, 5, 6,  ..., 8, 2, 6],
        [6, 5, 0,  ..., 4, 4, 0],
        ...,
        [2, 9, 4,  ..., 8, 8, 9],
        [5, 8, 0,  ..., 4, 9, 3],
        [4, 3, 9,  ..., 9, 5, 3]], device='cuda:0')

In [20]:
# seq, batch, embed
(logits := embedding(params=test_params, occupations=test_occupations))

tensor([[[-4.3741,  6.1671, 12.7498,  ...,  0.0000,  0.0000,  0.0000],
         [-3.4021,  4.7966,  9.9165,  ...,  0.0000,  0.0000,  0.0000],
         [-0.9720,  1.3705,  2.8333,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-0.9720,  1.3705,  2.8333,  ...,  0.0000,  0.0000,  0.0000],
         [-2.9161,  4.1114,  8.4998,  ...,  0.0000,  0.0000,  0.0000],
         [-0.9720,  1.3705,  2.8333,  ...,  0.0000,  0.0000,  0.0000]],

        [[-1.4580,  2.0557,  4.2499,  ...,  0.0000,  0.0000,  0.0000],
         [-2.4301,  3.4262,  7.0832,  ...,  0.0000,  0.0000,  0.0000],
         [-2.9161,  4.1114,  8.4998,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [-3.8881,  5.4819, 11.3331,  ...,  0.0000,  0.0000,  0.0000],
         [-0.9720,  1.3705,  2.8333,  ...,  0.0000,  0.0000,  0.0000],
         [-2.9161,  4.1114,  8.4998,  ...,  0.0000,  0.0000,  0.0000]],

        [[-2.9161,  4.1114,  8.4998,  ...,  0.0000,  0.0000,  0.0000],
         [-2.4301,  3.4262,  7.0832,  ...,  0

In [21]:
print(logits.shape)

torch.Size([105, 32, 48])


In [22]:
import einops as ein
from torch import nn

In [23]:
embed_dims = 16
key_dims = 8
L = 100
batch_dims = 32

dtype = torch.complex64

# Bring embedding vectors into key space for dot products
w_k = nn.Parameter(torch.randn(embed_dims, key_dims, dtype=dtype))
w_q = nn.Parameter(torch.randn(embed_dims, key_dims, dtype=dtype))

# Reeinterpret embedding vectors as semantic adjustments
w_v = nn.Parameter(torch.randn(embed_dims, embed_dims, dtype=dtype))

In [24]:
logits = torch.randn(L, batch_dims, embed_dims, dtype=dtype)
logits

tensor([[[ 0.2645-2.0310e-01j, -0.3343-7.7794e-01j, -0.0358+1.0639e+00j,
           ...,  0.4235+2.0753e-01j, -1.3245-6.8840e-01j,
           0.3364+2.6753e-01j],
         [ 0.4871-3.7249e-01j,  0.8095-4.5376e-01j, -0.7054+7.8988e-01j,
           ..., -0.6054+5.7797e-01j, -0.2716-1.0117e+00j,
          -0.8503+2.0867e-01j],
         [-0.1575+5.1280e-01j, -1.9658-7.5262e-01j, -0.0331-1.6130e-02j,
           ..., -0.8140-2.7404e-01j,  0.2139+2.8575e-02j,
           0.7865+3.6010e-01j],
         ...,
         [ 1.3511+8.1247e-01j, -0.8660+1.3701e+00j,  0.8380+1.1800e+00j,
           ..., -0.5017+1.3851e+00j,  0.8875-4.0611e-02j,
          -0.5202-7.7626e-01j],
         [ 0.1601-2.1490e-01j, -0.0124-6.9477e-01j, -0.0633-2.2570e-01j,
           ..., -0.5653-3.4111e-02j, -0.1430-1.3998e+00j,
          -1.0428-1.0925e-01j],
         [ 1.5891+5.1156e-02j, -0.8717-5.5777e-01j,  1.3570-6.7866e-01j,
           ...,  0.8542-9.4031e-01j, -0.2760-1.1877e+00j,
           1.1313-4.0418e-01j]],

      

In [25]:
q = torch.matmul(logits, w_q)
k = torch.matmul(logits, w_k)
v = torch.matmul(logits, w_v)

_q = ein.rearrange(q, "s b k -> b s k")
_kH = ein.rearrange(k, "s b k -> b k s").conj()
_v = ein.rearrange(v, "s b k -> b s k")
similarities = _q @ _kH

# TODO: this causes a reallocation
similarities = torch.softmax(torch.abs(similarities), dim=-1).to(dtype=dtype)
adjustments = similarities @ _v
adjustments

tensor([[[-2.3789e+00+6.2465j, -2.6606e-02+5.4006j,  1.5462e-01-4.3771j,
           ...,  3.2065e+00+1.8775j,  1.8957e+00+0.5284j,
          -2.3066e+00-4.1783j],
         [ 6.8465e+00-4.4575j, -2.0478e+00-2.6699j, -6.2038e-01+3.0587j,
           ..., -5.1668e+00+3.0779j, -2.0710e-01-0.0317j,
           7.7097e+00-1.5114j],
         [ 3.3653e+00-1.4593j, -4.0621e+00+3.4405j, -3.9628e+00+1.6167j,
           ..., -2.2917e-01-3.5143j, -2.4493e+00-0.3173j,
           1.5011e+00-0.5541j],
         ...,
         [ 2.1120e+00-0.4693j,  3.3200e+00+5.4233j, -3.9761e+00-3.4534j,
           ...,  3.9565e-01+6.3235j, -4.1536e+00-2.4587j,
          -2.3067e+00-0.7096j],
         [ 6.8465e+00-4.4575j, -2.0478e+00-2.6699j, -6.2038e-01+3.0587j,
           ..., -5.1668e+00+3.0779j, -2.0710e-01-0.0317j,
           7.7097e+00-1.5114j],
         [ 2.9308e+00-1.3930j, -3.5122e+00+2.8454j, -2.8409e+00+1.6719j,
           ...,  2.4966e-01-3.1353j, -2.1935e+00-0.8811j,
           1.7321e+00-0.0131j]],

      